In [1]:
!git clone https://github.com/AleksL04/ML-Project.git
%cd ML-Project

Cloning into 'ML-Project'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (68/68), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 68 (delta 31), reused 56 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (68/68), 27.48 KiB | 13.74 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/content/ML-Project


In [2]:
!git pull

Already up to date.


In [3]:
%%capture
!pip install --upgrade gensim
!gdown --folder 1rl_TJMHtcP-S0fTHNUpr9oi8Ndz2Y5z4 -O project_data
!gdown --folder 1RmC_We2lfsJontwrxlc4dzqEY_mNy-UW -O word2vec_model

In [4]:
ls -l

total 48
-rw-r--r-- 1 root root 32272 Dec 11 19:18 main_console.ipynb
drwxr-xr-x 2 root root  4096 Dec 11 19:18 model_code/
-rw-r--r-- 1 root root  1550 Dec 11 19:18 process_df.py
drwxr-xr-x 2 root root  4096 Dec 11 19:18 project_data/
drwxr-xr-x 2 root root  4096 Dec 11 19:18 word2vec_model/


In [5]:
import pandas as pd
import numpy as np

In [6]:
train_df = pd.read_csv('project_data/train.csv')
valid_df = pd.read_csv('project_data/valid.csv')
test_df = pd.read_csv('project_data/test.csv')

In [7]:
from process_df import process_df

X_train = process_df(train_df)
y_train = train_df['label'].values

X_valid = process_df(valid_df)
y_valid = valid_df['label'].values

X_test = process_df(test_df)
y_test = test_df['label'].values

In [8]:
from model_code.LSTM_code import trainLSTM

LSTM_model = trainLSTM(X_train, y_train, X_valid, y_valid)

test_loss, test_acc = LSTM_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8936 - loss: 0.2572
Test Accuracy: 0.8965


In [16]:
from model_code.CNN_code import trainCNN

CNN_model = trainCNN(X_train, y_train, X_valid, y_valid)

test_loss, test_acc = CNN_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.8679 - loss: 0.3016
Test Accuracy: 0.8685


In [18]:
val_pred_LSTM = LSTM_model.predict(X_valid)
val_pred_CNN = CNN_model.predict(X_valid)

X_meta_train = np.column_stack((val_pred_LSTM, val_pred_CNN))

y_meta_train = y_valid

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [19]:
from model_code.Blender_code import trainBlender

Blender_model = trainBlender(X_meta_train, y_meta_train)

Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - accuracy: 0.5551 - loss: 0.5379 - learning_rate: 0.0010
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6898 - loss: 0.5512 - learning_rate: 0.0010
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7545 - loss: 0.5274 - learning_rate: 0.0010
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7524 - loss: 0.5378 - learning_rate: 0.0010
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8133 - loss: 0.5031 - learning_rate: 0.0010
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8182 - loss: 0.4934 - learning_rate: 0.0010
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8236 - loss: 0.4934 - learning_rate: 0.0010
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8587 - loss: 0.4630 - learning_rate: 0.0010
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8478 - loss: 0.4640 - learning_rate: 0.0010
Epoch 10/100
23/23

In [20]:
test_pred_LSTM = LSTM_model.predict(X_test)
test_pred_CNN = CNN_model.predict(X_test)

X_meta_test = np.column_stack((test_pred_LSTM, test_pred_CNN))

test_loss, test_acc = Blender_model.evaluate(X_meta_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8992 - loss: 0.2596
Test Accuracy: 0.9006
